#  <center>ANÁLISIS DE REGRESIÓN 
## Proyecto Final de Semestre (Código de Ejecucción)
###  Berriel Rangel Abraham  (315079885)

El objetivo del trabajo es constuir un modelo de Regresión Lineal para predecir la cantidad de tarifa esperada por viajes de un conjunto de datos el cual contiene los registros de los viajes realizados en el año 2018.

Los datos utilizados en el presente trabajo fueron recopilados y proporcionados a la Comisión de taxis y limusinas de la ciudad de Nueva York "Taxi and Limousine Commission(TLC)" por proveedores de tecnología autorizados en virtud de los Programas de mejora de pasajeros de taxis "Taxicab & Livery Passenger Enhancement Programs" (TPEP / LPEP). 

Primeramente importaremos una muestra representatriva de la base de datos, sobre la cual se realizara su respectuva limpieza, análisis exploratorio e ingeniería de datos para obtener una predicción.

# 1. Librerias

In [ ]:
#pip install tabulate

In [ ]:
#librerias

# Tratamiento de datos
import numpy as np
import pandas as pd
from tabulate import tabulate
import matplotlib.pyplot as plt


# Preprocesado y modelado
from sklearn.model_selection import train_test_split


In [ ]:
import pandas as pd  # para los dataframes 
import pickle  #I/O of BLOBs
import scipy.stats as stat  # para los test estadisticos
from scipy.stats import pearsonr  #para los coeficiientes de correlación
from datetime import datetime  #timestamps
from collections import Counter  #para index
import itertools as it  #combinations and something else
import numpy as np  #python para trabajar con arreglos.
import matplotlib.pyplot as plt  #plots
import seaborn as sns  #otra libreria para plotear, pero más chida
import folium  # libreria para crear mapas 
import json  # leer los .json que necesitemos
import matplotlib  #
import warnings
import statsmodels as sm
matplotlib.style.use('ggplot')  #nice plots
from pandas.plotting import table


%matplotlib inline

# 2. Importar datos

Debido a que tenemos una enorme cantidad de datos (112 M) y nuestros recursos de software son limitados nos concentraremos en obtener una muestra representativa de los datos para tratar la información, dicha muestra es una muestra aleatoria estratificada, es decir, la muestra se realizara de acuerdo a una o mas variables categoricas definidas, las cuales estarán en las mismas proporciones respecto a la población total, los 112 millones de datos.

Para eso crearemos una fución que nos permita obtener una muestra estratificada, dicha función realiza su tarea leyendo el archivo por "chunks" y sacando una muestra estratificada para cada chunk, muestras que en conjunto serán los datos que utilizaremos para nuestro trabajo, también definimos una función para obtener la proporción de datos de cada estrato respecto a la población para verificar que tomamos una muestra proporcional.

In [ ]:
def muestreo_estratificado(file,N,variables, cs = None):
    muestra = pd.DataFrame([])
    for df in pd.read_csv(file, chunksize=cs):
        N = N
        m1=df.groupby(variables,group_keys=False).apply(lambda x: x.sample(int(np.rint(N*len(x)/len(df))))).sample(frac=1).reset_index(drop=True)
        muestra = muestra.append(m1)
    return muestra

In [ ]:
def proporciones(file, variables, cs = None):
    P = pd.DataFrame([])
    s=0
    for df in pd.read_csv(file, chunksize=cs):
        s = s + df.shape[0]
        P = P.append(df.groupby(variables)['VendorID'].count()) 
    P["col"]= "total"
    Proporciones = P.groupby('col').sum().T
    Proporciones["prop"] = Proporciones["total"]/s
    return Proporciones


Dado que nuestro objetivo no solo es tener información, si no que esa información sea signifcante, debemos tener una proporción equitativa de datos respecto a una variable de interés,para efectos de número de datos, tomaremos los ID de empresas de taxis, para que todo el sistema de los mismos este bien representado, mientras que si se quisiera una proporción representativa respecto a otra variable sería analogo a lo mostrado abajo.

In [ ]:
prop = proporciones('C:/Users/berri/Desktop/Arcade/nyc_taxi/2018_Yellow_Taxi_Trip_Data.csv','VendorID',cs=3000000)

In [ ]:
prop

In [ ]:
prop.info()

Aquí nos indican que habría que tomar 41.9% de la empresa 1, 57.6618% de la empresa 2 y 4.381% de la empresa 4 (sin embargo sabemos que solo existen 2 empresas de taxis de NY, por lo cual es un indicio que hay inconsistencias que habrá que arreglar)

## 2.1.- Reconocimiento de datos 

In [ ]:
df = muestreo_estratificado('C:/Users/berri/Desktop/Arcade/nyc_taxi/2018_Yellow_Taxi_Trip_Data.csv',10000, "VendorID", 3000000)

In [ ]:
df = pd.read_csv(r"C:/Users/berri/Desktop/chido.csv")

In [ ]:
df.head()

Veamos que nuestros datos se componen de 17 columnas las cuales las podemos diferenciar de dos tipos distintos, variables categoricas y variables cuantitativas, entre estas se encuentran:
#### - variables categoricas: 
        * VendorID
        * RatecodeID
        * store_and_fwd_flag
        * PULocationID
        * DOLocationID
        * payment_type
#### - variables cuantitativas: 
        * tpep_pickup_datetime 	
        * tpep_dropoff_datetime 
        * passenger_count 	
        * trip_distance
        * PULocationID 	
        * DOLocationID 	
        * payment_type 	
        * fare_amount 	
        * extra 	
        * mta_tax 	
        * tip_amount 	
        * tolls_amount 	
        * improvement_surcharge 	
        * total_amount

Podemos ver que las variables categoricas nos muestran las columnas que que se relacionaran los catalogos de nuestro modelo relacional, ademas las variables cuantitativas, tpep_pickup_datetime, tpep_dropoff_datetime son columnas con fechas, estas requeriran un tratamiento especial, las variables cuantitativas restantes se analizaran buscando anomalias en los datos.

In [ ]:
df.info()

Abajo podemos observar que esta muestra que tenemos es representativa respecto a las proporciones que antes nos fueron dadas:


In [ ]:
df.VendorID.value_counts()

In [ ]:
df['VendorID'].value_counts() / len(df['VendorID'])

# 3.- Tratamiento de datos crudos, anomalías y outliers

## 3.1 .- Conversión a datatime de datos y creación de "duración de viaje"

In [ ]:
df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"], format= '%m/%d/%Y %I:%M:%S %p' )


In [ ]:
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"], format= '%m/%d/%Y %I:%M:%S %p' )


In [ ]:
(df['tpep_dropoff_datetime'].dt.month).value_counts()

In [ ]:
(df['tpep_dropoff_datetime'].dt.year).value_counts()

In [ ]:
print('Anterior tamaño: %d' % len(df))
df = df[df["tpep_dropoff_datetime"].dt.year==2018]
print('Nuevo tamaño: %d' % len(df))

Podemos observar que respecto a los meses, nuestro catálogo de datos tambien está bien distribuido respecto al número de meses que toma, a la par que creamos una nueva columna bastante importante


In [ ]:
df['trip_duration_min'] = round(((df['tpep_dropoff_datetime']-df['tpep_pickup_datetime']).dt.seconds)/60, 1)


In [ ]:
df["trip_duration_min"]

## 3.2 .- Tratamiento de anomalías respecto a VendorID

In [ ]:
sns.catplot(x="VendorID", y="trip_duration_min",kind="strip",data=df)


En el gráfico de arriba podemos notar inmediatamente dos valores que hay que erradicar:
#### - Viajes con duración de hasta 1200 min 
#### - Existencia de una cuarta empresa de Taxi NYC

In [ ]:
df['VendorID'].hist(bins=100)


Que si bien no es de gran número respecto a las otras dos empresas , lo mejor es eliminarlas

In [ ]:
print('Anterior tamaño: %d' % len(df))
df = df[df.VendorID<4]
print('Nuevo tamaño: %d' % len(df))

## 3.3 .- Distacia de viaje iguales a cero y tiempo de viaje 

In [ ]:
sns.distplot(df['trip_duration_min'],kde=False)
plt.title('Distribution Duration')
plt.show()

In [ ]:
sns.distplot(df['trip_distance'],kde=False)
plt.title('Distribution Duration')
plt.show()

Esto deja en claro que en muchas ocasiones los histogramas están muy lejos de proveer una buena información acerca de los outliers

In [ ]:
dist_0 = df[df["trip_distance"]==0]
my_plot = dist_0.plot("trip_distance", "passenger_count", kind="scatter")
plt.show()

In [ ]:
print('Anterior tamaño: %d' % len(df))
df = df[df.trip_distance>0]
print('Nuevo tamaño: %d' % len(df))

Verificamos que realmente hayan desaparecido

In [ ]:
dist_0 = df[df["trip_distance"]==0]
my_plot = dist_0.plot("trip_distance", "passenger_count", kind="scatter")
plt.show()

In [ ]:
df["trip_duration_min"].max()

In [ ]:
# plot histogram of fare
df[df.trip_duration_min<1439.9].trip_duration_min.hist(bins=10000, figsize=(14,3))
plt.xlabel('Duration in minutes')
plt.title('Histogram');

In [ ]:
df["trip_duration_min"].mean()

Tenemos presencia muy importante de outliers para la parte de la duración de viajes. Por lo que consideramos que a partir de 3 horas podemos consideras que los viajes son anomalías dado que incluso con outliers la media es 30 min por viaje. 

In [ ]:
print('Tamaño anterior: %d' % len(df))
df = df[df.trip_duration_min<180]
print('Nuevo tamaño: %d' % len(df))

In [ ]:
df["trip_duration_min"].max()

In [ ]:
# plot histogram of fare
df[df.trip_duration_min<1798].trip_duration_min.hist(bins=10000, figsize=(14,3))
plt.xlabel('Duración en minutos')
plt.title('Histograma');

## 3.3 Numero de pasajeros igual a cero y mayores a 6.

Primeramente para esta parte nos dimos cuenta que existian registros en los cuales existen viajes con cero pasajeros regitrados, dado que esto es una inconsistencia decididos eliminar dichos registros, ademas, la normatividad que regula estos taxis indica que la cantidad máxima de pasajeros permitidos en un taxi amarillo por ley es de cuatro, en un taxi de cuatro pasajeros o de cinco pasajeros en un taxi de cinco pasajeros, excepto que se debe aceptar un pasajero adicional si tal El pasajero es menor de siete años y está sostenido en el regazo de un pasajero adulto sentado en la parte trasera. Por dicha razón se decidio eliminar lor registros con pasajeros cero y mayores a 6.

La informacion mencionada en el parrafo anterior se puede verificar en la pagina 33 de la siguente fuente:

https://www1.nyc.gov/assets/tlc/downloads/pdf/rule_book_current_chapter_54.pdf 

In [ ]:
sns.boxplot(df['passenger_count'], orient='horizontal')
plt.title('Box Plot de los pasajeros')


In [ ]:
sns.boxplot(df['total_amount'], orient='horizontal')
plt.title('Box Plot de las tarifas')

In [ ]:
print('Tamaño anterior: %d' % len(df))
df = df[df.total_amount<200]
print('Nuevo tamaño: %d' % len(df))

In [ ]:
sns.distplot(df['total_amount'],kde=False)
plt.title('Disribución del Pago')
plt.show()

In [ ]:
print('Tamaño anterior: %d' % len(df))
df = df[df.total_amount>0]
print('Nuevo tamaño: %d' % len(df))

In [ ]:
sns.boxplot(df['total_amount'], orient='horizontal')
plt.title('Box Plot de las tarifas')

In [ ]:
pasajeros_0 = df[df["passenger_count"]<=0]
my_plot = pasajeros_0.plot("passenger_count", "total_amount", kind="scatter")
plt.show()

In [ ]:
pasajeros_0 = df[df["passenger_count"] > 6]
my_plot = pasajeros_0.plot("passenger_count", "total_amount", kind="scatter")
plt.show()

In [ ]:
print('Tamaño anterior: %d' % len(df))
df = df[df.passenger_count>0]
print('Nuevo tamaño: %d' % len(df))

In [ ]:
print('Tamaño anterior: %d' % len(df))
df = df[df.passenger_count<=6]
print('Nuevo tamaño: %d' % len(df))

In [ ]:
df["passenger_count"].max()

In [ ]:
df["passenger_count"].min()

##  3.4 Montos totales iguales a cero

Para este paso nos dimos cuenta que existian viajes con monto total igual a cero pero con numero de pasajeros y distancia registrados, debido a que eso representa una inconsistencia procedemos a eliminar dichas filas.

In [ ]:
monto_0 = df[df["total_amount"]==0]
my_plot = monto_0.plot("total_amount", "trip_distance", kind="scatter")
plt.show()

In [ ]:
print('Tamaño anterior: %d' % len(df))
df = df[df.total_amount>0]
print('Nuevo tamaño: %d' % len(df))

## 3.5 Verificar que no existan datos blancos o vacíos

In [ ]:
df.isna().sum().plot(kind="bar")
data = df.dropna()

In [ ]:
del data

In [ ]:
print(df.columns.values)

   ###### - VendorID:
        * El valor maximo de esta columna es 4, es decir que existen valores inconsistentes con respecto al
        catalogo.

De acuerdo al grafico anterior correpondiente a VendorID, de su frecuencia y distribución podemos ver que los registros en los cuales su VendorID es 4 representan una proporcion casi nula de los datos, ademas respecto a las columnas de total_amount y PULocationID, los registros con dato VendorID igual a 4, representan proporciones insignificantes comparadas con los datos de VendorID diferentes.

###### - passenger_count:
        * Esta columna es cuantitativa pero por su rango tan pequeño lo consideraremos en esta sección para 
        nuestro analisis
        * El valor minimo de esta columna es 0, es decir, existen viajes realizados pero que no registraron 
        ningun pasajero.
        * Deacuerdo a la fuente de informacion debemos considerar pautas establecias para el tratamiento de 
        esta columna

###    Variables cuantitativas


Ahora analizaremos la frecuencia y distribucion de las variables cuantitativas para visualizar las columnas y observaciones a tratar en la  limpieza de los datos.

  Analizando la tabla y las graficas anteriores llegamos a las siguientes observaciones respecto a estas columnas:
   
   ###### - total_amount:
        * Esta columna es el resultado de la suma de las columnas de fare_amount, extra, mta_tax, tip_amount,
        tolls_amount, improvement_surcharge.
        * El valor minimo es negativo, por lo que podemos pensar que existen valores negativos en las celdas que
        conforman el resultado de total_amount.
   ###### - trip_distance:
        * El valor minimo de la distancia del viaje es cero, es decir, existen viajes cobrados pero que en la 
        base de datos no recorrieron distancia alguna.
   ###### - extra:
        * El mayor volumen de estos registros se encuentran en los valores 0, 0.50. 1)

  ###### - total_amount:
        * Esta columna es el resultado de la suma de las columnas de fare_amount, extra, mta_tax, tip_amount,
        tolls_amount, improvement_surcharge.
        * El valor minimo es negativo, por lo que podemos pensar que existen valores negativos en las celdas que
        conforman el resultado de total_amount.
   ###### - trip_distance:
        * El valor minimo de la distancia del viaje es cero, es decir, existen viajes cobrados pero que en la 
        base de datos no recorrieron distancia alguna.
   ###### - RatecodeID:
        * El valor maximo de esta columna es 99, es decir que existen valores inconsistentes con respecto al
        catalogo

In [ ]:
(df['passenger_count']).value_counts()

In [ ]:
#Crer catalgos

In [ ]:
print('Old size: %d' % len(df))
df = df[df.trip_distance<150]
print('New size: %d' % len(df))

In [ ]:
df.trip_distance.max()

## 3.6 .-Carga de código de distritos para mejor análisis

In [ ]:
zone_lookup = pd.read_csv("/Users/berri/Desktop/Arcade/taxi_zone_lookup.csv")

### Sincronizar columnas con el cargo anterior de datos

In [ ]:
df.rename(columns={'PULocationID':'LocationID'}, inplace=True)


# 4.-Análisis Exploratorio Estádistico

Antes de plantear un modelo de regresión creemos oportuno no dejar pasar la oportunidad de hacer un completo análisis de demás rubros que podrían ser de igual intereses que una regresión del total amount para una empresa de Taxis para poder hacer ciertas tomas de decisiones con base en lo expuesto en está parte del escrito

## 4.1 .- ¿En qué época del año se utilizan más los taxis? 


¿En qué época del año se utilizan más los taxis? Creemos una gráfica que, para cada mes, muestre el número promedio de viajes registrados cada día. Debido a las diferencias entre las zonas de Nueva York, queremos visualizar la misma información para cada distrito.

Estamos interesados en las recolecciones en lugar de la entrega, por lo que usamos solo la columna 'tpep_pickup_datetime' para nuestro análisis y 'PULocationID' para unir el conjunto de datos con un mapeo entre las identificaciones y el municipio asociado.


Las columnas iniciales son:

tpep_pickup_datetime Contiene la marca de tiempo exacta en el formato estándar% Y-% m-% d% H:% M:% S.

PULocationID Es una identificación única de la ubicación, que se utilizará para unirse al marco de datos que contiene los distritos.

El marco de datos resultante está compuesto por dos columnas: una contiene la marca de tiempo de la recolección y la otra contiene el municipio en el que se realiza la recolección.

In [ ]:
 df1 = df[['tpep_pickup_datetime', 'LocationID']]
df1 = df1.merge(zone_lookup[['LocationID', 'Borough']], how='inner' ,on='LocationID') #Inner join entre las columnas de interes
df1 = df1[['tpep_pickup_datetime', 'Borough']]
#df1 = df1.loc[df['tpep_pickup_datetime'].dt.year == 2018].fillna(0)
#display(df1.head())

In [ ]:
df1.info()

Veamos si nuestra muestra contiene datos suficientes de todos los meses del año para poder realizar el análisis 

In [ ]:
(df1['tpep_pickup_datetime'].dt.month).value_counts()


Después del fragmento, tenemos los datos limpios, ¡así que podemos ejecutar nuestras operaciones en eso!

## 4.1.1- Ingeniería de datos:
Para cada operación que hemos realizado aquí hay una ampliación de por qué realizamos esa operación.

## 4.1.2.-Bucle anidado: 
Para cada distrito y para cada mes, tenemos que dividir el número de recogidas para los días del mes para obtener el promedio diario, por lo que con "set (df ['Borough'])" tenemos todos los distritos y con "for i in range (1, max (df ['pickup']. dt.month) +1)" iteramos con todos los meses.

Para evitar errores construimos una lista de los días del mes en los que el elemento en la posición 0 corresponde a Janaury, el elemento en la posición 1 corresponde a Febraury y así sucesivamente.

tmp y tmpdf: En el bucle for anidado hay un marco de datos temporal que almacena las capturas en el mes que estamos analizando, podemos evitarlo, pero en este momento preferimos la legibilidad en lugar de las actuaciones, liberaremos la memoria cada uno y cada vez que pasa el bucle.

Usamos vectorización porque pandas está optimizado para operaciones vectoriales.

In [ ]:
max(df1['tpep_pickup_datetime'].dt.month)

In [ ]:
df12 = pd.DataFrame()
tmp = pd.Series()
months = [31,28,31,30,31,30,31,31,30,31,30,31]
for borough in set(df1['Borough']): #
    for i in range(1, max(df1['tpep_pickup_datetime'].dt.month)):
        tmpdf = df1.loc[(df1['tpep_pickup_datetime'].dt.month == i) & (df1['Borough'] == borough)]['tpep_pickup_datetime']
        if len(tmpdf) != 0:
            tmp = tmp.append(pd.Series(tmpdf.count()/months[i-1]), ignore_index=True)
    df12[borough] = tmp
    del tmp
    del tmpdf
    tmp = pd.Series()
df12.rename(index ={0:'Enero', 1:'Febr', 2:'Mar', 3:'Abr', 4:'May', 5:'Jun', 6:'Jul', 7:'Ago',8:'Sept', 9:'Oct', 10:'Nov',11:'Dic'}, inplace = True)
df12 = df12.fillna(0)
display(df12)

In [ ]:
df12

## 4.1.3.- Gráficas

In [ ]:
df12.plot.bar(figsize=(20,20),subplots=True,layout=(5,2))


## 4.1.4.- Obtención de datos diarios 

In [ ]:
df['pickup_day']=df['tpep_pickup_datetime'].dt.day_name()
df['dropoff_day']=df['tpep_dropoff_datetime'].dt.day_name()

In [ ]:
figure,ax=plt.subplots(nrows=2,ncols=1,figsize=(10,10))
sns.countplot(x='pickup_day',data=df,ax=ax[0])
ax[0].set_title('Número de viajes solicitados en cada día de la semana')
sns.countplot(x='dropoff_day',data=df,ax=ax[1])
ax[1].set_title('Número de viajes concluidos en cada día de la semana')
plt.tight_layout()

In [ ]:
import datetime


In [ ]:
def timezone(x):
    if x>=datetime.time(4, 0, 1) and x <=datetime.time(10, 0, 0):
        return 'Mañana'
    elif x>=datetime.time(10, 0, 1) and x <=datetime.time(16, 0, 0):
        return 'Medio día'
    elif x>=datetime.time(16, 0, 1) and x <=datetime.time(22, 0, 0):
        return 'Atardecer'
    elif x>=datetime.time(22, 0, 1) or x <=datetime.time(4, 0, 0):
        return 'Noche'
    
df['pickup_timezone']=df['tpep_pickup_datetime'].apply(lambda x :timezone(datetime.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S').time()) )
df['dropoff_timezone']=df['tpep_dropoff_datetime'].apply(lambda x :timezone(datetime.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S').time()) )

In [ ]:
figure,ax=plt.subplots(nrows=2,ncols=1,figsize=(10,10))
sns.countplot(x='pickup_timezone',data=df,ax=ax[0], edgecolor=sns.color_palette("dark", 3), linewidth=20)
ax[0].set_title('La frecuencia de viajes solicitados en franjas horarias')
sns.countplot(x='dropoff_timezone',data=df,ax=ax[1], edgecolor=sns.color_palette("dark", 3), linewidth=20)
ax[1].set_title('La frecuencia de viajes terminados en franjas horarias')
plt.tight_layout()

In [ ]:
figure,ax=plt.subplots(nrows=1,ncols=2,figsize=(10,5))
df['pickup_hour']=df['tpep_pickup_datetime'].dt.hour
df.pickup_hour.hist(bins=24,ax=ax[0])
ax[0].set_title('Distribución de la duración de viajes')
df['dropoff_hour']=df['tpep_dropoff_datetime'].dt.hour
df.dropoff_hour.hist(bins=24,ax=ax[1])
ax[1].set_title('Distribución de las dejadas')

In [ ]:
plt.title('BoxPlot de la duración de los viajes en minutos')
sns.boxplot(df['trip_duration_min'], orient='horizontal')


In [ ]:
bins=np.array([0,5,15,30,60,150])
df['duration_time']=pd.cut(df.trip_duration_min,bins,labels=["< 10", "11-25", "25-40","40-60",">60"])

In [ ]:
ax1=df.groupby('pickup_day')['duration_time'].value_counts(normalize=True).unstack()
ax1.plot(kind='bar', stacked='True')
plt.title('La distribucion del porcentaje de las diferentes duraciones de los viajes')

In [ ]:
figure,ax=plt.subplots(nrows=1,ncols=3,figsize=(15,5))
ax1=df[(df.duration_time !="< 10")].groupby('pickup_day')['duration_time'].count()
ax1.plot(kind='bar',ax=ax[0])
ax[0].set_title('Frecuencia de viajes por dia')
ax2=df[(df.duration_time !="< 10")].groupby('pickup_day')['duration_time'].value_counts(normalize=True).unstack()
ax2.plot(kind='bar', stacked='True',ax=ax[1])
ax[1].set_title('Porcentaje de viajes en distintas franjas horarias')
ax3=df[(df.duration_time !="< 10")].groupby('pickup_day')['duration_time'].value_counts().unstack()
ax3.plot(kind='bar',ax=ax[2])
ax[2].set_title('Comparación de horarios')


## 4.1.5 .- Conclusiones para RQ1
Notamos que Manhattan tiene una gran cantidad de viajes promedio diarios, también observamos que la distribución es una especie de uniforme) para la mayoría de los distritos.

Si cambiamos la escala del promedio diario (de lo contrario, Manhattan tomará toda la densidad) podemos ver que el mes con más conteos es Janaury.

Con una breve charla con una amiga, me dijo que Manhattan tiene esta enorme cantidad de taxis amarillos con respecto a otros distritos porque una ley no permite entrada de taxis verdes.

In [ ]:
###

# 4.2.- ¿Cuáles son las franjas horarias con más pasajeros?

## 4.2.1 Planteamiento
    ¿Cuáles son las franjas horarias con más pasajeros? Establezca sus propios intervalos de tiempo y descubra cuáles son aquellos en los que los taxis conducen el mayor número de pasajeros en general en Nueva York y repita el análisis para cada distrito.

In [ ]:
df2 = df[['LocationID', 'tpep_pickup_datetime', 'passenger_count']]
df2 = df2.merge(zone_lookup[['LocationID', 'Borough']], how='inner' ,on='LocationID').fillna(0) # Inner Join entre las columnas interes y nuestro previo csv
df2 = df2.drop(['LocationID'], axis=1) #Tirar tablas inutiles

In [ ]:
trips = {0:"1 to 6", 1:"1 to 6", 2:"1 to 6", 3:"1 to 6", 4: "1 to 6", 5: "1 to 6", 6: "1 to 6", 
         7:"7 to 11", 8:"7 to 11", 9:"7 to 11", 10:"7 to 11", 11:"7 to 11", 
         12:"12 to 14", 13:"12 to 14", 14: "12 to 14", 
         15:"15 to 19", 16:"15 to 19", 17: "15 to 19", 18:"15 to 19", 19:"15 to 19",
         20:"20 to 22", 21: "20 to 22", 22: "20 to 22", 
         23:"23 to 24"}

In [ ]:
df2['tpep_pickup_datetime'] = df2['tpep_pickup_datetime'].dt.hour


In [ ]:
df2['tpep_pickup_datetime'] = df2['tpep_pickup_datetime'].map(trips)


In [ ]:
df2.head()


In [ ]:
df22 = df2.groupby(['Borough','tpep_pickup_datetime'])
df23 = df22.count()


In [ ]:
df23.groupby('Borough').plot.bar()


In [ ]:
####

## 4.3 Distribuciones de los viajes por distrito 
    En esta sección nos concentramos en desallorar una ingeniería de datos capaz de poder agrupar por distritos todos los diferentes viajes realizados en ellos y con esa información poder nosotros ajustar una función de distribución para que esa misma información sea de utilidad al tratar de desarollar otro enfoque de ánalisis estadístico sobre el mismo conjunto de datos

In [ ]:
df3 = df[['LocationID', 'DOLocationID', 'trip_duration_min']]


In [ ]:
df3.dropna(inplace=True) 
df3.rename(columns= {'LocationID':'PULocationID'}, inplace=True)

#merge df3 y zone_lookup para el vecindario de donde se tomo el taxi
trips = df3.merge(zone_lookup[['LocationID', 'Borough']], how='inner',
                      left_on='PULocationID',
                      right_on='LocationID').fillna("")[['trip_duration_min','PULocationID','DOLocationID','Borough']]

trips.rename(columns= {'Borough':'PUBorough'}, inplace=True)

# merrge entre trips y vecindarios para ver donde se bajo el taxi
trips = trips.merge(zone_lookup[['LocationID', 'Borough']], how='inner',
                      left_on='DOLocationID',
                      right_on='LocationID').fillna("")[['trip_duration_min','PULocationID','DOLocationID','PUBorough','Borough']]

trips.rename(columns= {'Borough':'DOBorough'}, inplace=True)


#vemos los viajes para saber si esto funciono bien 
display(trips.head())

In [ ]:
trips.drop(trips.index[np.where(trips['trip_duration_min'] <= 0)], inplace=True) #tiramos si la duracion de viajes es 0 o mneos 
display(len(trips[trips['trip_duration_min']>200])) #y tambien para ver tiramos si hay outliers 

In [ ]:
trips

In [ ]:

# conjunto de todos los vecindarios 
Boroughs = list(set(zone_lookup['Borough']))
Boroughs[Boroughs.index('Staten Island')] = 'StatenIsland'
del Boroughs[Boroughs.index('Unknown')] 
#liberamos memorio
Bronx = trips[(trips['PUBorough']==trips['DOBorough']) & (trips['PUBorough']=='Bronx')]['trip_duration_min']
Brooklyn = trips[(trips['PUBorough']==trips['DOBorough']) & (trips['PUBorough']=='Brooklyn')]['trip_duration_min']
EWR = trips[(trips['PUBorough']==trips['DOBorough']) & (trips['PUBorough']=='EWR')]['trip_duration_min']
Manhattan = trips[(trips['PUBorough']==trips['DOBorough']) & (trips['PUBorough']=='Manhattan')]['trip_duration_min']
Queens = trips[(trips['PUBorough']==trips['DOBorough']) & (trips['PUBorough']=='Queens')]['trip_duration_min']
StatenIsland = trips[(trips['PUBorough']==trips['DOBorough']) & (trips['PUBorough']=='StatenIsland')]['trip_duration_min']
SwitchedBorough = trips[trips['PUBorough']!=trips['DOBorough']]['trip_duration_min']
#SwitchedBorough puede ser especificada o llamada como encontrar la los viajes más frecuentes entre 2 vecindario

In [ ]:
SwitchedBorough

In [ ]:
df31 = pd.DataFrame()
for B in Boroughs:
    df32 = eval("pd.DataFrame({'Duration':" + B +",'Borough':['"+str(B)+"']*len("+B+")})")
    df31 = pd.concat([df31, df32])

df31 = pd.concat([df31, pd.DataFrame({'Duration':SwitchedBorough,'Borough':['SwitchedBorough']*len(SwitchedBorough)})])

    
sns.set(style="whitegrid")

sns.boxenplot(x="Borough", y="Duration",
              color="b", order= Boroughs+['SwitchedBorough'],
              scale="linear", data=df31)

Obviamente, se puede ver que hay algunos valores atípicos en los datos, especialmente para Manhattan, Bronx, Brooklyn y Queens. Generalmente, los valores atípicos están por encima de 500 minutos. Podemos ignorarlos ya que tenemos demasiados datos. También podemos observar la distribución de datos de manera más efectiva, y para esto ocuparemos una técnica estadístca llamada Z-Score


En estadística, la puntuación estándar es el número de desviaciones estándar por las que el valor de una puntuación bruta (es decir, un valor observado o un punto de datos) está por encima o por debajo del valor medio de lo que se está observando o midiendo. Los puntajes brutos por encima de la media tienen puntajes estándar positivos, mientras que los que están por debajo de la media tienen puntajes estándar negativos.

Se calcula restando la media de la población de una puntuación bruta individual y luego dividiendo la diferencia por la desviación estándar de la población. Este proceso de convertir una puntuación bruta en una puntuación estándar se denomina estandarización o normalización (sin embargo, "normalizar" puede referirse a muchos tipos de proporciones; consulte normalización para obtener más información: https://en.wikipedia.org/wiki/Standard_score


## 4.3.1 Verificación formal usando z-score

In [ ]:

Bronx_z = np.abs(stat.zscore(Bronx))
Bronx.drop(Bronx.index[np.where(Bronx_z > 3)], inplace=True) #np.where(Bronx_z > 3)
Brooklyn_z = np.abs(stat.zscore(Brooklyn))
Brooklyn.drop(Brooklyn.index[np.where(Brooklyn_z > 3)], inplace=True) 

EWR_z = np.abs(stat.zscore(EWR))
EWR.drop(EWR.index[np.where(EWR_z > 3)], inplace=True)

Manhattan_z = np.abs(stat.zscore(Manhattan))
Manhattan.drop(Manhattan.index[np.where(Manhattan_z > 3)], inplace=True)

Queens_z = np.abs(stat.zscore(Queens))
Queens.drop(Queens.index[np.where(Queens_z > 3)], inplace=True)

StatenIsland_z = np.abs(stat.zscore(StatenIsland))
StatenIsland.drop(StatenIsland.index[np.where(StatenIsland_z > 3)], inplace=True)

SwitchedBorough_z = np.abs(stat.zscore(SwitchedBorough))
SwitchedBorough.drop(SwitchedBorough.index[np.where(SwitchedBorough_z > 3)], inplace=True)

In [ ]:
df_wo_outliers = pd.DataFrame()
for B in Boroughs:
    df2 = eval("pd.DataFrame({'Duration':" + B +",'Borough':['"+str(B)+"']*len("+B+")})")
    df_wo_outliers = pd.concat([df_wo_outliers, df2])
df_wo_outliers = pd.concat([df_wo_outliers,
                            pd.DataFrame({'Duration':SwitchedBorough,'Borough':['SwitchedBorough']*len(SwitchedBorough)})])

#check visually again;

sns.set(style="whitegrid")

sns.boxenplot(x="Borough", y="Duration",
              color="b", order= Boroughs+['SwitchedBorough'],
              scale="linear", data=df_wo_outliers)


Ahora está más claro. Según el diagrama de caja, Bronx tiene la mayor cantidad de valores atípicos. Al igual que Manhattan,  Queens solo tiene un valor atípico. La mediana de la duración del viaje es similar para Manhattan, Bronx, Brooklyn y Queens. Además, en Manhattan, la gente tiende a hacer más viajes cortos. Los datos de SwitchedBorough se difunden más porque, por lo general, lleva más tiempo viajar fuera de la ciudad. Por eso también tiene muchos valores atípicos.

### 4.3.2 Aproximación de la distribución empírica a la teoríca 

Empíricamente intentaremos ajustar alguna distribución a nuestros datos para cada distrito. Entonces, encontrarán los mejores parámetros para la distribución especificada. Se utilizará la suma del error cuadrado (SSE) cuando se comparen las distribuciones entre sí. Por tanto, la distribución con SSE mínima será la que mejor se ajuste.

In [ ]:
#Encontramos la mejor distribucion que se le ajusta a los datos

def bestDist(data, bins=200):
    #Obtenemos el histograma de los dtaos originales 
    y, x = np.histogram(data,bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0 # dado que hay 200 bins, los separamos por igual en el eje X
                   
    #En Scipy, existen muchas distribuciones pero aqui solo selecciono las que suelen modelar mejor estos eventos:
    DISTRIBUTIONS = [ stat.beta,stat.cauchy,stat.chi2,stat.expon,stat.exponnorm,stat.gamma,stat.laplace,stat.logistic,
                     stat.lognorm,stat.norm,stat.pareto,stat.rayleigh,stat.uniform ]
    # La que mejor se ajusto ; solo para corroborar, le asignamos algunas muestras
    best_distribution = stat.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf
        
    #Probaremos todas las distribuciones de arriba para estimar los parametros de la distribucion de los datos originales
    for dist in DISTRIBUTIONS:
         # tratamos de hacer encajar la distribucion
        try:
            # Ignoramos la advertencia de los datos no pueden ser ajustados ("outliers")
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')
                # fit dist to data
                prms = dist.fit(data)  #fit nos ayuda a encontrar los mejores parametros a la distribucion
                #Separamos las partes de los parametros tales como, la localizacion, escala y kurtosis, si es que existen
                arg = prms[:-2]
                loc = prms[-2]
                scale = prms[-1]
        
             #Después de haber encontrado los mejores parametros de la distribucion especificada, calcularemos la 
                #densidad ajustada y veremos la suma de errores al cuadrado"""
                pdf = dist.pdf(x, loc=loc, scale=scale, *arg) #it generates pdf from x; remember that we generate x from our original data to fit the histogram
                sse = np.sum(np.power(y - pdf, 2)) #sum of square error. this is our decision criteria
                # Si se ajusto al modelo, que lo plotee
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                    end
                except Exception:
                    pass
                # Si se encuentra un mejor párametro con menor error al cuadrado, que lo sustituya 
                if best_sse > sse > 0:
                    best_dist = dist
                    best_prms = prms
                    best_sse = sse
        
        except Exception:
            pass
    return (best_dist.name, best_prms)

#Guardemos la mejor distribucion y sus parametros y usemoslos para generar la pdf.
    
def pdf(best_dist, best_prms, size=1000):
    # Separate parts of parameters
    arg = best_prms[:-2]
    loc = best_prms[-2]
    scale = best_prms[-1]

      #empecemos con la grafica 
    "ppf se refiere a la funcion de distribucion inversa (tambien llamada de percentiles)"

    start = best_dist.ppf(0.001, *arg, loc=loc, scale=scale) if arg else best_dist.ppf(0.001, loc=loc, scale=scale)
    end = best_dist.ppf(0.999, *arg, loc=loc, scale=scale) if arg else best_dist.ppf(0.999, loc=loc, scale=scale)

    # Ahora creamos la pdf
    x = np.linspace(start, end, size) #ya debemos de saber que np.linspace nos da un array que contiene numeros entre a,b de tamaño size
    y = best_dist.pdf(x, loc=loc, scale=scale, *arg) 
    pdf = pd.Series(y, x)
    return pdf

Lo probamos para todos los distritos



### 4.3.3 Queens 

In [ ]:
# Queens
Qns_best_fit_name, Qns_best_fit_prms = bestDist(Queens, bins=200)
Qns_best_dist = getattr(stat, Qns_best_fit_name) #getattr() nos regresa el valor del atributo nombrado del objeto

Qns_pdf = pdf(Qns_best_dist, Qns_best_fit_prms)

Qns_prm = list(map(lambda x: "%.3f" % x, Qns_best_fit_prms)) #for the title round the parameters

#Creamos el histograma y le sobreponemos una pdf
sns.set(style="white", palette="muted", color_codes=True)

fig, ax = plt.subplots()
Qns_pdf.plot(lw=2, label='pdf', legend=True, ax=ax)
ax2 = ax.twinx()
sns.distplot(Queens, bins=50,kde=False, color="g", ax=ax2)

plt.xlabel(u'Duración (min)')
plt.ylabel("Frecuencia")
plt.title(u'El mejor ajuste de distribución para Queens\n'+Qns_best_fit_name+' con parámetros \n'  +'('+Qns_prm[0]+','+Qns_prm[1]+')') # 
plt.show(fig)

### 4.3.4 Brooklyn 

In [ ]:
# Brooklyn
Bklyn_best_fit_name, Bklyn_best_fit_prms = bestDist(Brooklyn, bins=200)
Bklyn_best_dist = getattr(stat, Bklyn_best_fit_name) #getattr() nos regresa el valor del atributo nombrado del objeto

Bklyn_pdf = pdf(Bklyn_best_dist, Bklyn_best_fit_prms)

Bklyn_prm = list(map(lambda x: "%.3f" % x, Bklyn_best_fit_prms)) #for the title round the parameters

#Creamos el histograma y le sobreponemos una pdf
sns.set(style="white", palette="muted", color_codes=True)

fig, ax = plt.subplots()
Bklyn_pdf.plot(lw=2, label='pdf', legend=True, ax=ax)
ax2 = ax.twinx()
sns.distplot(Brooklyn, bins=50,kde=False, color="g", ax=ax2)

plt.xlabel(u'Duración (min)')
plt.ylabel("Frecuencia")
plt.title(u'El mejor ajuste de distribución para Brooklyn\n'+Bklyn_best_fit_name+' con parámtros \n' 
          +'('+Bklyn_prm[0]+','+Bklyn_prm[1]+','+Bklyn_prm[2]+')') # You can comment this line out if you don't need title
plt.show(fig)


Dado que los datos de Brooklyn disminuyen repentinamente hasta los 15 minutos de duración del viaje, obtuvimos una distribución exponencialmente normal, también se denomina distribución gaussiana modificada exponencialmente (EMG). Podemos decir que Brooklyn está a punto de encajar con más datos

Los datos de EWR y Staten Island no son demasiados. Así que no es necesario estimar su distribución. Por ejemplo, EWR son 108 observaciones y 78 para Staten Island

### 4.3.5 Bronx 

In [ ]:
# Bronx
Brnx_best_fit_name, Brnx_best_fit_prms = bestDist(Bronx, bins=100)
Brnx_best_dist = getattr(stat, Brnx_best_fit_name) #getattr() nos regresa el valor del atributo nombrado del objeto

Brnx_pdf = pdf(Brnx_best_dist, Brnx_best_fit_prms)

Brnx_prm = list(map(lambda x: "%.3f" % x, Brnx_best_fit_prms)) #for the title round the parameters

#Creamos el histograma y le sobreponemos una pdf
sns.set(style="white", palette="muted", color_codes=True)

fig, ax = plt.subplots()
Brnx_pdf.plot(lw=2, label='pdf', legend=True, ax=ax)
ax2 = ax.twinx()
sns.distplot(Bronx, bins=50,kde=False, color="g", ax=ax2)

plt.xlabel(u'Duración (min)')
plt.ylabel("Frecuencia")
plt.title(u'El mejor ajuste de distribución para Bronx\n'+Brnx_best_fit_name+' con parámetros \n' 
          +'('+Brnx_prm[0]+','+Brnx_prm[1]+','+Brnx_prm[2]+')') # You can comment this line out if you don't need title
plt.show(fig)


El último es Cambios de Vecindario. Eso significa que examinaremos si el taxi va de un distrito a otro. Esperamos que la duración de los viajes sea larga todo el tiempo. Por lo tanto, lo más probable es que obtengamos una distribución de cola pesada como lognormal.

### 4.3.6 Cambios de Vecindario

In [ ]:
# Cambios de Vecindario
SBoro_best_fit_name, SBoro_best_fit_prms = bestDist(SwitchedBorough, bins=200)
SBoro_best_dist = getattr(stat, SBoro_best_fit_name) #getattr() nos regresa el valor del atributo nombrado del objeto 

SBoro_pdf = pdf(SBoro_best_dist, SBoro_best_fit_prms)

SBoro_prm = list(map(lambda x: "%.3f" % x, Bklyn_best_fit_prms)) #Generamos el titulo con los parámetros expuestos 

#Creamos el histograma y le sobreponemos una pdf
sns.set(style="white", palette="muted", color_codes=True)

fig, ax = plt.subplots()
SBoro_pdf.plot(lw=2, label='pdf', legend=True, ax=ax)
ax2 = ax.twinx()
sns.distplot(SwitchedBorough, bins=50,kde=False, color="g", ax=ax2)

plt.xlabel(u'Duración (min)')
plt.ylabel("Frecuencia")
plt.title(u'EL mejor ajuste de distribución cuando el  viaje es entre dos distritos\n'+SBoro_best_fit_name+' con parámetros \n' 
          +'('+SBoro_prm[0]+','+SBoro_prm[1]+','+SBoro_prm[2]+')') # You can comment this line out if you don't need title
plt.show(fig)

### 4.3.7 Manhattan

In [ ]:
# Manhattan;
Man_best_fit_name, Man_best_fit_prms = bestDist(Manhattan, bins=200)
Man_best_dist = getattr(stat, Man_best_fit_name) #getattr() nos regresa el valor del atributo nombrado del objeto
Man_pdf = pdf(Man_best_dist, Man_best_fit_prms)
Man_prm = list(map(lambda x: "%.3f" % x, Man_best_fit_prms)) #Generamos el titulo con los parámetros expuestos
#Creamos el histograma y le sobreponemos una pdf
sns.set(style="white", palette="muted", color_codes=True)
fig, ax = plt.subplots()
Man_pdf.plot(lw=2, label='pdf', legend=True, ax=ax)
ax2 = ax.twinx()
sns.distplot(Manhattan, bins=50,kde=False, color="g", ax=ax2)
plt.xlabel(u'Duración (min)')
plt.ylabel("Frecuencia")
plt.title(u'El mejor ajuste de distribución para Manhattan\n'+Man_best_fit_name+' con parámetros \n' 
          +'('+Man_prm[0]+','+Man_prm[1]+','+Man_prm[2]+')') # You can comment this line out if you don't need title
plt.show(fig)

Conclusiones para RQ3
Dado que los viajes se acumulan en algún momento, alrededor de 25 minutos, obtuvimos una distribución exponencialmente normal. Eso tiene sentido. Debido a que es fácil de adivinar si cambia de distrito, tomará más tiempo. Entonces no hay muchos datos para viajes cortos. Además, exponencialmente normal también se ajusta bien a los datos.

Por lo tanto, generalmente obtuvimos una distribución sesgada correcta cuando el taxi está en el mismo municipio. Los datos están sesgados a la derecha porque, en general, la gente hace viajes cortos en Nueva York. Sin embargo, si cambia de municipio, debería acumularse. En promedio, toma media hora.

In [ ]:
# No ejecutes esto hasta que acabes.
del df3


## 4.4.- Tipos de Pago más frecuente 

¿Cuál es la forma de pago más común? Descubramos la forma en que se ejecutan los pagos en cada municipio y visualizemos el número de pagos por cualquier medio posible. Luego, ejecutaremos la prueba Chi-cuadrado para ver si el método de pago está correlacionado con el municipio..


La única columna que necesitamos es 'payment_type' y, como es habitual, 'PULocationID' para unirse a los distritos.

En lugar de usar la función 'del', asignamos el puntero de las variables temporales a una nueva lista, el recolector de basura hace el resto del trabajo.

Para datos más comprensibles usamos un mapa (diccionario) que para cada PaymentID asocia el método de pago correspondiente, esto será útil porque podemos vectorizar la operación de convertir payment_type en una cadena más comprensible.

El marco de datos resultante finalmente se compone de dos columnas: una contiene el método de pago del viaje como una cadena y la otra contiene el municipio en el que se realizó la recogida.

Rellenamos los NA con 0 para que en el proceso de ingeniería de datos podamos simplemente omitirlos con un uso inteligente de los diccionarios de Python

In [ ]:
number_to_payment = {1: "Credit Card",2: "Cash",3: "No Charge", 4: "Dispute",5: "Unknown",6: "Voided Trip"}


In [ ]:

df4 = df[['payment_type', 'LocationID']]
# Recordemos que Zone_lookup (fue el csv que te pase) es lo que nos permite tener un mapeo entre LocationID y Borough (Borough)
df4 = df4.merge(zone_lookup[['LocationID', 'Borough']], how='inner' ,on='LocationID').fillna(0) #Inner join between the interested columns and the previous dataset
df4 = df4[['payment_type', 'Borough']]
df4['payment_type'] = df4['payment_type'].map(number_to_payment)
display(df4.head())

###### -Ingeniería de datos:
    Necesitamos construir un marco de datos que para cada municipio almacene el recuento de métodos de pago, por lo que usamos nuevamente funciones vectorizadas para acelerar las actuaciones. El resultado del siguiente fragmento es un marco de datos con los distritos en las filas y los métodos de pago en las columnas.

In [ ]:
#Aquí solo es para crear el dataframe vacío donde guardaremos los datos
df42 = pd.DataFrame(columns=[el for el in set(df4['Borough'])],  index=[el for el in set(df4['payment_type'])])

#for every borough
for key in set(df4['Borough']):
    #Agrupar registros en un dataframe vacío y contar, el cambio de nombre puede ser obsoleto pero por razones de seguridad podemos dejarlo aquí
    df42[key] = df4.loc[df4['Borough'] == key].groupby('payment_type').count().rename(index=str, columns={'Borough':key})

df42 = df42.T.fillna(0)
display(df42)

###### Gráfica 
    Es interesante trazar los resultados del marco de datos y su transposición para obtener más información sobre lo que está sucediendo

In [ ]:
df42.plot(kind='bar')
df42.T.plot(kind = 'bar')


In [ ]:
df4["Borough"].value_counts()

In [ ]:
df

In [ ]:
df1["Borough"].value_counts()

###### $\chi^2$ Test:
       Usamos una función de construcción de la biblioteca scipy para calcular la prueba estadística definida de la siguente manera:
<center> $\chi^2 =\sum_{i=1}^{n}\frac{(O_i - E-_i)^2}{E_i}$


In [ ]:
stat.chi2_contingency(df42)


De acuerdo con la teoría de la prueba $ \chi^2$, no se rechaza la hipótesis nula y la prueba dice que el método de pago está altamente correlacionado con el municipio.
La forma más común de pagar un taxi en Nueva York es con la tarjeta de crédito, en segundo lugar está el efectivo.
Hay tres distritos sin datos suficientes para ver en la trama lo que está sucediendo, y son: "EWR," Staten Island "y" Bronx ".

In [ ]:
del df4
del df42


## 4.6 .- Correlación entre la distancia del viaje
¿Se correlaciona una larga distancia con la duración del viaje en promedio? Haz una gráfica que muestre la dependencia entre la distancia y la duración del viaje. Luego calcule el coeficiente de Pearson, ¿es significativo? Comenta los resultados que obtengas.

In [ ]:
df5 = df[['tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance', 'trip_duration_min']]
display(df5.head())

Limpie aquellas filas que están incluidas en los valores cero de 'trip_distance', esto es para evitar valores atípicos.

In [ ]:
df5=df5[(df5.trip_distance > 0) & (df5.trip_distance < 100) ]


In [ ]:
df5['trip_duration_hou']= df5['trip_duration_min'] / 60 #df.apply(getRowDuration, axis=1)

In [ ]:
plt.scatter(x = df5['trip_distance'], y = df5['trip_duration_hou'], c = df5['trip_distance'] + df5['trip_duration_hou'], alpha= 0.75)
plt.xlabel("Trip Distance")
plt.ylabel("Duration")
plt.title("Scatter_Plot")
plt.show()

In [ ]:
corr, p_value = pearsonr(df5['trip_distance'], df5['trip_duration_hou'])
print ("The Pearson correlation coefficient calculated with scipy is: " + str("%0.2f"%corr))
print ("The Pearson correlation coefficient calculated with pandas is: " + str("%0.2f"%df5['trip_distance'].corr(df5['trip_duration_hou'])))

In [ ]:
## CRQ1

In [ ]:
df

In [ ]:
print('Old size: %d' % len(df))
df = df[df.trip_distance>0]
print('New size: %d' % len(df))

In [ ]:
corr, p_value = pearsonr(df5['trip_distance'], df5['trip_duration_hou'])
print ("The Pearson correlation coefficient calculated with scipy is: " + str("%0.2f"%corr))
print ("The Pearson correlation coefficient calculated with pandas is: " + str("%0.2f"%df5['trip_distance'].corr(df5['trip_duration_hou'])))

## 4.6.- ¿La tarifa por milla cambia en el distrito de Nueva York? ¶ 


   
        ¿La tarifa por milla cambia en el distrito de Nueva York? Queremos descubrir si los gastos de un usuario que disfruta de Taxis en una zona es diferente a los que lo utiliza en otra.

In [ ]:
dfcrq1 = df[['trip_distance', 'fare_amount', 'LocationID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_duration_min']].fillna(0)
     
#after the nested loop i have to perform some manipulations in order to have the complete the dataframe 
#zone_lookup = pd.read_csv("/Users/ince/Desktop/uni/adm/hws/hw2/taxi_zone_lookup.csv") # this dataset have a map between the LocationID and Borough
dfcrq1 = dfcrq1.merge(zone_lookup[['LocationID', 'Borough']], how='inner' ,on='LocationID').fillna(0) #Inner join between the interested columns and the previous dataset
dfcrq1 = dfcrq1.loc[(dfcrq1['trip_distance'] > 0) & #removing the < 0 mile-distance records
                    (dfcrq1['fare_amount'] > 0) & 
                    (dfcrq1['tpep_pickup_datetime'].dt.year == 2018) &
                    (dfcrq1['tpep_dropoff_datetime'].dt.year == 2018) &
                    (dfcrq1['trip_distance'] < 100)].reset_index() #removing the > 100 mile-distance records

dfcrq1 = dfcrq1.drop(['LocationID', 'index'], axis=1) #drop useless columns

dfcrq1['$/mile'] = dfcrq1['fare_amount'] / dfcrq1['trip_distance'] #calc the dollar per mile amount
dfcrq1['$*minutes/mile'] = dfcrq1['$/mile']*dfcrq1['trip_duration_min']
display(dfcrq1.head())

In [ ]:
new_york_dollar_per_mile_mean = dfcrq1['$/mile'].mean()
new_york_dollar_per_mile_std = dfcrq1['$/mile'].std()

In [ ]:
labels, values = zip(*Counter(dfcrq1['$/mile'].round(decimals=1)).items())

indexes = np.arange(len(labels))
width = 1.7
plt.xlim(0,400)

plt.bar(indexes, values, width)
plt.xlabel('$/mile')
plt.ylabel('Frequency')
plt.show()

Considerations

We see that in the distribution there's some outlier, in the next section we want to evaluate if they're really outliers or maybe some traffic issues. Also the distribution is not Gaussian so in theory we cannot run the t-test on this data.

In [ ]:
mean = dfcrq1.groupby('Borough').mean()['$/mile'] #this groups by borough, calc the mean and takes only the column that i want to plot
std = dfcrq1.groupby('Borough').std()['$/mile'] #same as above, but for the standard deviaton
dfcrq12 = pd.DataFrame([mean, std], columns=[el for el in set(dfcrq1['Borough'])],  index=['mean', 'std']) #combine the previous result in a dataframe
dfcrq12['NY'] = pd.Series([new_york_dollar_per_mile_mean, new_york_dollar_per_mile_std], index = ['mean', 'std'])
display(dfcrq12)

In [ ]:
dfcrq12.plot(kind='bar')
dfcrq12.T.plot(kind='bar')

In [ ]:
dfcrq1.head()

In [ ]:
dfcrq13 = pd.DataFrame()
combs = list(it.combinations(set(dfcrq1['Borough']), 2))
for b1,b2 in combs:
    ttest_result = stat.ttest_ind(dfcrq1.loc[dfcrq1['Borough'] == b1]['$/mile'], dfcrq1.loc[dfcrq1['Borough'] == b2]['$/mile'])
    dfcrq13[b1+'-'+b2] = pd.Series([ttest_result[0], ttest_result[1]], index=['ttest', 'pval'])
display(dfcrq13)

In [ ]:
x = range(len(set(dfcrq13)))
x_labels = set(dfcrq13)
y = [dfcrq13[i][0] for i in x_labels]

markerline, stemlines, baseline = plt.stem(x,y, '-.')
plt.setp(baseline, color='orchid', linewidth=1)
plt.title('TTest')
plt.xticks(x, x_labels, rotation='vertical')
plt.show()

y = [dfcrq13[i][1] for i in x_labels]

markerline, stemlines, baseline = plt.stem(x,y, '-.')
plt.setp(baseline, color='orchid', linewidth=1)
plt.title('p-val')
plt.xticks(x, x_labels, rotation='vertical')
plt.show()

## 4.8.- Visualización geográfica


    ¡Visualicemos los movimientos de los taxis! NYC está dividida en muchas zonas de taxis. Para cada viaje en taxi amarillo conocemos la zona en la que el Taxi recoge y deja a los usuarios. Visualicemos, en un mapa de chropleth, el número de viajes que comienzan en cada zona. Luego, haz otro mapa para contar las carreras que terminan en la zona única.

In [ ]:
dfcrq2 = df[['LocationID', 'DOLocationID']]
geo_data = json.load(open(r'C:/Users/berri/Desktop/Arcade/nyu-2451-36743-geojson.json'))                          
display(dfcrq2.head())


In [ ]:

NY=folium.Map(
    location=[40.7142700, -74.0059700],   #coordinates of new York
    zoom_start=11,                        
    tiles='CartoDB positron'              #style of our map
)
NY

In [ ]:
folium.GeoJson(geo_data,style_function=lambda feature: {'fillColor':'blue','color' : 'orchid','weight' : 1,'fillOpacity' : 0.5, 'colorOpacity' : 0.5 }).add_to(NY)
NY

In [ ]:
NY.save(outfile= "test.html")

In [ ]:
df 

In [ ]:
location_id_to_number_of_pickups=dfcrq2.groupby('LocationID')['LocationID'].count()
location_id_to_number_of_dropoff=dfcrq2.groupby('DOLocationID')['DOLocationID'].count()

location_id_to_number_of_pickups.head()

df_zone_to_pickup_to_dropoff=pd.DataFrame(index=list(range(1,266)),columns=[])
df_zone_to_pickup_to_dropoff['ZoneID']=list(range(1,266))

zone_index_to_dropoff_number = []
zone_index_to_pickup_number = []

for i in range(1,266):
    if i in location_id_to_number_of_pickups:
        zone_index_to_pickup_number.append(location_id_to_number_of_pickups[i])
    else:
        zone_index_to_pickup_number.append(0)    
        
    if i in location_id_to_number_of_dropoff:
        zone_index_to_dropoff_number.append(location_id_to_number_of_dropoff[i])
    else:
        zone_index_to_dropoff_number.append(0)     #Necesitamos esto para verificar que no haya 0 (puede haber 0 taxis en la zona)

df_zone_to_pickup_to_dropoff['taxi_pickup'] = zone_index_to_pickup_number
df_zone_to_pickup_to_dropoff['taxi_dropoff'] = zone_index_to_dropoff_number
display(df_zone_to_pickup_to_dropoff.head())

In [ ]:
from folium import plugins
from folium.plugins import HeatMap

In [ ]:
NY2 = folium.Map(
    location=[40.7142700, -74.0059700],   #coordinates of new York
    zoom_start=11,                        
    tiles='CartoDB positron'              #style of our map
)

NY2.choropleth(
    geo_data=geo_data,  #our geojson datas
    data=df_zone_to_pickup_to_dropoff,    #our dataframe
    columns=['ZoneID', "taxi_pickup"],
    key_on='feature.properties.objectid', #the key in geojson file that way want to take as zone
    fill_color='YlGnBu',   #the color scale that we want
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name='Tipos de pago en el 2018',
    highlight=True    #enable the highlight function, to enable highlight functionality when you hover over each area.
)

folium.Marker(
    location=[40.7730135746, -73.8702298524],
    popup='Airport LaGuardia',
    icon=folium.Icon(icon='plane')
).add_to(NY2)

folium.Marker(
    location=[40.6413111, -73.7781391],
    popup='John F. Kennedy International Airport',
    icon=folium.Icon(icon='plane')
).add_to(NY2)

folium.Marker(
    location=[40.7828647, -73.9653551],
    popup = 'Central Park',
    icon=folium.Icon(color='red')
).add_to(NY2)

folium.Marker(
    location=[40.758895, -73.985131],
    popup = 'Times Square',
    icon=folium.Icon(color='red')
).add_to(NY2)

folium.Marker(
    location=[40.7061927, -74.0091604],
    popup = 'Wall Street',
    icon=folium.Icon(color='red')
).add_to(NY2)

folium.Marker(
    location=[40.692013, -74.181557],
    popup='Newark Liberty International Airport',
    icon=folium.Icon(icon='plane')
).add_to(NY2)

NY2


In [ ]:
NY2 = folium.Map(
    location=[40.7142700, -74.0059700],   #coordinates of new York
    zoom_start=11,                        
    tiles='CartoDB positron'              #style of our map
)

NY2.choropleth(
    geo_data=geo_data,  #our geojson datas
    data=df_zone_to_pickup_to_dropoff,    #our dataframe
    columns=['ZoneID', 'taxi_dropoff'],
    key_on='feature.properties.objectid', #the key in geojson file that way want to take as zone
    fill_color='YlGnBu',   #the color scale that we want
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name='Number of taxi taken in 2018',
    highlight=True    #enable the highlight function, to enable highlight functionality when you hover over each area.
)

folium.Marker(
    location=[40.7730135746, -73.8702298524],
    popup='Airport LaGuardia',
    icon=folium.Icon(icon='plane')
).add_to(NY2)

folium.Marker(
    location=[40.6413111, -73.7781391],
    popup='John F. Kennedy International Airport',
    icon=folium.Icon(icon='plane')
).add_to(NY2)

folium.Marker(
    location=[40.7828647, -73.9653551],
    popup = 'Central Park',
    icon=folium.Icon(color='red')
).add_to(NY2)

folium.Marker(
    location=[40.758895, -73.985131],
    popup = 'Times Square',
    icon=folium.Icon(color='red')
).add_to(NY2)

folium.Marker(
    location=[40.7061927, -74.0091604],
    popup = 'Wall Street',
    icon=folium.Icon(color='red')
).add_to(NY2)

folium.Marker(
    location=[40.692013, -74.181557],
    popup='Newark Liberty International Airport',
    icon=folium.Icon(icon='plane')
).add_to(NY2)

NY2


In [ ]:
NY3 = folium.Map(
    location=[40.7142700, -74.0059700],   #coordinates of new York
    zoom_start=11,                        
    tiles='CartoDB positron'              #style of our map
)

NY3.choropleth(
    geo_data=geo_data,
    data=df_zone_to_pickup_to_dropoff,
    columns=['ZoneID', 'taxi_dropoff'],
    key_on='feature.properties.objectid',
    fill_color='YlGnBu',
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name='Number of taxi drops in 2018',
    highlight=True    
)
folium.Marker(
    location=[40.7730135746, -73.8702298524],
    popup='LaGuardia Airport',
    icon=folium.Icon(icon='plane')
).add_to(NY3)

folium.Marker(
    location=[40.6413111, -73.7781391],
    popup='John F. Kennedy International Airport',
    icon=folium.Icon(icon='plane')
).add_to(NY3)

folium.Marker(
    location=[40.7828647, -73.9653551],
    popup = 'Central Park',
    icon=folium.Icon(color='red')
).add_to(NY3)

folium.Marker(
    location=[40.758895, -73.985131],
    popup = 'Times Square',
    icon=folium.Icon(color='red')
).add_to(NY3)

folium.Marker(
    location=[40.7061927, -74.0091604],
    popup = 'Wall Street',
    icon=folium.Icon(color='red')
).add_to(NY3)

folium.Marker(
    location=[40.692013, -74.181557],
    popup='Newark Liberty International Airport',
    icon=folium.Icon(icon='plane')
).add_to(NY3)

NY3

In [ ]:
## En que Vecindarios se paga con que tipo 

In [ ]:
dfcrq3 = df[['payment_type', 'LocationID']]
dfcrq3[dfcrq3.duplicated(subset=['LocationID'],keep=False)]
dfcrq3.payment_type.value_counts()


In [ ]:
z = dfcrq3.groupby(["LocationID"])['payment_type'].apply(lambda x: x.mode()[0])


In [ ]:
z = z.to_frame()

In [ ]:
z['LocationID'] = z.index


In [ ]:
z["payment_type"].value_counts()


In [ ]:
z

In [ ]:
NY4 = folium.Map(
    location=[40.7142700, -74.0059700],   #coordinates of new York
    zoom_start=11,                        
    tiles='CartoDB positron'              #style of our map
)

NY4.choropleth(
    geo_data=geo_data,  #our geojson datas
    data=z,    #our dataframe
    columns=['LocationID', "payment_type"],
    key_on='feature.properties.objectid', #the key in geojson file that way want to take as zone
    fill_color='YlGnBu',   #the color scale that we want
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name='Tipos de pago en el 2018',
    highlight=True    #enable the highlight function, to enable highlight functionality when you hover over each area.
)

folium.Marker(
    location=[40.7730135746, -73.8702298524],
    popup='Airport LaGuardia',
    icon=folium.Icon(icon='plane')
).add_to(NY2)

folium.Marker(
    location=[40.6413111, -73.7781391],
    popup='John F. Kennedy International Airport',
    icon=folium.Icon(icon='plane')
).add_to(NY2)

folium.Marker(
    location=[40.7828647, -73.9653551],
    popup = 'Central Park',
    icon=folium.Icon(color='red')
).add_to(NY2)

folium.Marker(
    location=[40.758895, -73.985131],
    popup = 'Times Square',
    icon=folium.Icon(color='red')
).add_to(NY2)

folium.Marker(
    location=[40.7061927, -74.0091604],
    popup = 'Wall Street',
    icon=folium.Icon(color='red')
).add_to(NY2)

folium.Marker(
    location=[40.692013, -74.181557],
    popup='Newark Liberty International Airport',
    icon=folium.Icon(icon='plane')
).add_to(NY2)

NY4


In [ ]:
NY4.save(outfile= "test.html")

In [ ]:
NY3.save(outfile= "test1.html")

In [ ]:
NY2.save(outfile= "test2.html")

# 5.- Regresión múltiple: $$\mathbf{Y} = \mathbb{X} \underline{\beta} + {\varepsilon}$$

## 5.1.- Creación de modelo

### 5.1.1.- Indicio de dependencia lineal

In [ ]:
sns.scatterplot(x='trip_distance',y='total_amount',data=df)


### 5.1.- Creación de columna "mes"

In [ ]:
df['pickup_month']=df['tpep_pickup_datetime'].dt.month
df['dropoff_month']=df['tpep_dropoff_datetime'].dt.month

In [ ]:
data2=df.loc[:,['passenger_count','total_amount','trip_duration_min', 'pickup_day', 'dropoff_day', 'pickup_month',
   'dropoff_month','pickup_timezone','dropoff_timezone','trip_distance',"store_and_fwd_flag"]]

In [ ]:
df['log_distance']=np.log(df.trip_distance)
df['log_fare_amount']=np.log(df.fare_amount)
sns.scatterplot(x='log_distance',y='log_fare_amount',data=df)

### 5.1.2.- Elección de Variables que conformaran la matriz $\mathbb{X}$

In [ ]:
data2=df.loc[:,['passenger_count','total_amount','trip_duration_min', 'pickup_day', 'dropoff_day', 'pickup_month',
   'dropoff_month','pickup_timezone','dropoff_timezone','log_distance','trip_distance',"store_and_fwd_flag"]]

### 5.1.3 Creación de Variables Categorícas (Dummies)

In [ ]:
data2=pd.get_dummies(data2,columns=['pickup_day',"passenger_count",'dropoff_day','pickup_month','dropoff_month','pickup_timezone', 'dropoff_timezone'])

In [ ]:
data2

### 5.1.4.- Matriz de Correlación 

In [ ]:
data2.corr()

In [ ]:
sns.heatmap(data2.corr());


### 5.1.4 .- Creación de matriz $\mathbb{X}$

In [ ]:
data2.columns

In [ ]:
base_line_col=['trip_distance']
predictor_cols=['passenger_count_1',"passenger_count_2",'trip_distance',
               'pickup_day_Friday','pickup_day_Monday','pickup_day_Saturday','pickup_day_Sunday',
               'pickup_day_Thursday','pickup_day_Tuesday','pickup_day_Wednesday','dropoff_day_Friday',
               'dropoff_day_Monday','dropoff_day_Saturday','dropoff_day_Sunday','dropoff_day_Thursday',
               'dropoff_day_Tuesday','dropoff_day_Wednesday','pickup_month_1','pickup_month_5','pickup_month_6',
               'dropoff_month_1','dropoff_month_5','dropoff_month_6','pickup_timezone_Noche',
               'pickup_timezone_Medio día','pickup_timezone_Mañana','dropoff_timezone_Atardecer',
               'dropoff_timezone_Noche','dropoff_timezone_Medio día','dropoff_timezone_Mañana',"trip_duration_min"]
target_col=['total_amount']

## 5.2 Importación del modelo desde sklearn

In [ ]:
from sklearn import  metrics
from sklearn.model_selection import cross_val_score
def modelfit(estimator,data_train,data_test,predictors,target):
    #print(data_train.head())
    #fitting model
    estimator.fit(data_train[predictors],data_train.loc[:,target])
    score = estimator.score(data_train[predictors],data_train.loc[:,target])
    
    #train data prediction
    train_pred=estimator.predict(data_train[predictors])
    #cross_validation score
    cv_score=cross_val_score(estimator,data_train[predictors],data_train.loc[:,target],cv=20,scoring='neg_mean_squared_error')
    
    cv_score=np.sqrt(np.abs(cv_score))
    #Print model report:
    print ("\nModel Report")
    print ("RMSE on Train Data: %.4g" % np.sqrt(metrics.mean_squared_error(data_train.loc[:,target].values, train_pred)))
    print ("CV Score : Mean - %.4g | Std - %.4g | Min - %.4g | Max - %.4g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
    
    test_pred=estimator.predict(data_test[predictors])
    print ("RMSE on Test Data: %.4g" % np.sqrt(metrics.mean_squared_error(data_test.loc[:,target].values, test_pred)))
    
    print(score)
    
    
    return test_pred, score

### 5.2.1 Configuración de datos de entramiento y de validación 

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
validation_size = 0.70

X_train, X_test = train_test_split(data2,test_size=validation_size, random_state=42)

In [ ]:
from math import sqrt

In [ ]:
mean_pred=np.repeat(X_train[target_col].mean(),len(X_test[target_col]))
from sklearn.metrics import mean_squared_error as mae
sqrt(mae(X_test[target_col],mean_pred))

In [ ]:
alg1 = LinearRegression(normalize=True)
print('The baseline model')
y_pred=modelfit(alg1, X_train, X_test,base_line_col,target_col)
coef1 = alg1.coef_
print('The coeffient is {}'.format(coef1))

In [ ]:
alg2 = LinearRegression(normalize=True)
y_pred=modelfit(alg2, X_train, X_test, predictor_cols,target_col)
coef1 = pd.Series(alg2.coef_[0], predictor_cols).sort_values()
coef1.plot(kind='bar', title='Model Coefficients')

In [ ]:
import statsmodels.api as sm
import statsmodels as sms
from sklearn import linear_model

In [ ]:
regr = linear_model.LinearRegression()

In [ ]:
regr.fit(X_train[predictor_cols], X_train.loc[:,target_col])

In [ ]:

print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)


In [ ]:
X = sm.add_constant(X_train[predictor_cols])


In [ ]:
model = sm.OLS(X_train.loc[:,target_col],X_train[predictor_cols]).fit() 
predictions = model.predict(X_train[predictor_cols]) 
 
print_model = model.summary()
print(print_model)

In [ ]:
model.resid

In [ ]:
model.resid.hist(bins=80)


In [ ]:
print('mean=%.3f stdv=%.3f' % (np.mean(model.resid), np.std(model.resid)))

In [ ]:
from fitter import get_distributions


In [ ]:
from fitter import Fitter
f = Fitter(model.resid,distributions=['norm'])
f.fit()

In [ ]:
f.summary()


In [ ]:
f.fitted_param

In [ ]:
from scipy.stats import shapiro


In [ ]:
stat, p = shapiro(model.resid_pearson)

In [ ]:
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
	print('La muestra se ve Gaussiana ( No Rechazamos H0)')
else:
	print('La muestra no se ve Gaussiana (Rechazamos H0)')

In [ ]:
from scipy.stats import anderson


In [ ]:
result = anderson(model.resid_pearson)


In [ ]:
print('Statistic: %.3f' % result.statistic)

In [ ]:
for i in range(len(result.critical_values)):
	sl, cv = result.significance_level[i], result.critical_values[i]
	if result.statistic < result.critical_values[i]:
		print('%.3f: %.3f, data looks normal (fail to reject H0)' % (sl, cv))
	else:
		print('%.3f: %.3f, data does not look normal (reject H0)' % (sl, cv))

In [ ]:
from scipy import stats

k2, p = stats.normaltest(model.resid_pearson)
alpha = 0.05
print("p = {:g}".format(p))

In [ ]:
if p < alpha:  # null hypothesis: x comes from a normal distribution
    print("The null hypothesis can be rejected")
else:
    print("The null hypothesis cannot be rejected")

In [ ]:
model.resid_pearson

In [ ]:
import scipy.stats as stats
>>> fig = sm.qqplot(model.resid, stats.t, distargs=(4,))
>>> plt.show()

In [ ]:
fig = sm.qqplot(model.resid, stats.norm, fit=True, line="45")
plt.show()

In [ ]:
>>> fig = sm.qqplot(model.resid, stats.norm, loc=0, scale=1)
>>> plt.show()

In [ ]:
from statsmodels.graphics.gofplots import qqplot
from matplotlib import pyplot

In [ ]:
qqplot(model.resid, line='s')
pyplot.show()

In [ ]:
predictions

In [ ]:
type(predictions)

In [ ]:
X_train["total_amount"]

In [ ]:
validation = pd.concat([X_train["total_amount"], predictions],axis=1)

In [ ]:
validation.reset_index=True

In [ ]:
validation

In [ ]:
validation

In [ ]:
#model = sm.OLS(X_test.loc[:,target_col],X_test[predictor_cols]).fit() 
predictions = model.predict(X_test[predictor_cols]) 
 
print_model = model.summary()
print(print_model)

In [ ]:
(predictions)

In [ ]:
(X_test["total_amount"])

In [ ]:
model.fittedvalues

In [ ]:
X_test["total_amount"]

In [ ]:
predictions

In [ ]:
model.fittedvalues

In [ ]:
plt.figure(figsize=(10, 6), dpi=120, facecolor='w', edgecolor='b')
f = range(0,len(model.resid))
k = [0 for i in range(0,len(model.resid))]
plt.scatter( f, model.resid, label = 'residuals')
plt.plot( f, k , color = 'red', label = 'regression line' )
plt.xlabel('fitted points ')
plt.ylabel('residuals')
plt.title('Residual plot')
plt.legend()